In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint


In [3]:
df = pd.read_csv('../data/data1/logvRNA.csv')
patients = df.patient.unique()
df

,patient,dpi,log_vRNA
0,CHID46,13,4.037825
1,CHID46,16,5.033826
2,CHID46,20,5.903090
3,CHID46,23,5.964858
4,CHID46,27,5.531990
...,...,...,...
62,CHID08,19,6.750628
63,CHID08,24,6.811993
64,CHID08,26,6.360983
65,CHID08,31,4.420137


In [4]:
from sklearn.metrics import mean_squared_error
def beta(t, tau, k,beta_0,beta_ifty):
    if t <= tau:
        return beta_0
    else:
        return beta_ifty + (beta_0-beta_ifty)*np.exp(-k*(t-tau))
def model(y,t, beta_0,beta_ifty,k,dlt,p,d,tau):
    T = y[0]
    I = y[1]
    V = y[2]
    
    c=23
    
    b = beta(t, tau, k,beta_0,beta_ifty)
    
    dTdt = d*(10**4) - d * T - b*T*V
    dIdt = b*T*V-dlt*I
    dVdt = p*I-c*V
    
    dydt = [dTdt,dIdt,dVdt]
    return dydt    

def pred_LV(t, beta_0,beta_ifty,k,dlt,p,d,tau,times):
    T_0 = 10**4  # ml
    I_0 = 0
    V_0 = 10**-3
    h = 0.01
    
    sol = odeint(model,[T_0,I_0,V_0],t, args =(beta_0,beta_ifty,k,dlt,p,d,tau))   
    
    pred = np.log10([x[2] for x in sol])  #  sol[:,2])    
    return np.array([pred[int(x/h)] for x in times])

def error(params):
    beta_0 = params[0]
    beta_ifty = params[1]
    k = params[2]
    dlt = params[3]
    p = params[4]
    d = params[5]
    tau = params[6]  
    
    t = np.arange(0,80,0.01)
    
    dg = df.groupby('patient').get_group(patients[0])
    
    pred = pred_LV(t,beta_0,beta_ifty,k,dlt,p,d,tau, dg.dpi.values)
    true = dg['log_vRNA'].values

    
    
    
    return round(mean_squared_error(pred, true),5)
    
    

In [5]:
xx1 = [0.409E-6, 0.233E-6, 0.249, 0.775, 14.5E3, 0.03, 7];
xx2 = [2.0209E-7, 1.2719E-7, 0.41184, 1.52453, 41085.62299, 0.03901, 6];
xx3 = [1.9975e-07, 1.265e-07, 0.33022,    1.5147,    40734,    0.039136,    6.0109];

xx4 = [2.0273E-7, 1.2499E-7, 1.81286, 1.57038, 42987.62471, 0.03935, 7];
xx5 = [2.0262e-07, 1.2506e-07,    1.7837,    1.5725,    42997,    0.039383,    7.0061]

M3 = [xx1,xx2,xx3,xx4,xx5];
M3

[[4.09e-07, 2.33e-07, 0.249, 0.775, 14500.0, 0.03, 7],
 [2.0209e-07, 1.2719e-07, 0.41184, 1.52453, 41085.62299, 0.03901, 6],
 [1.9975e-07, 1.265e-07, 0.33022, 1.5147, 40734, 0.039136, 6.0109],
 [2.0273e-07, 1.2499e-07, 1.81286, 1.57038, 42987.62471, 0.03935, 7],
 [2.0262e-07, 1.2506e-07, 1.7837, 1.5725, 42997, 0.039383, 7.0061]]

In [6]:
error(xx4)

0.0045

In [7]:
M = [[0 for j in range(8)] for i in range(5)]

for i in range(5):
#     M[i][0] = error(M1[i])
#     M[i][1] = error(M2[i])
    M[i][0] = error(M3[i])
    M[i][1:] = M3[i]
    
dff = pd.DataFrame(M,index=['paper', 'Berkeley Madonna 1','fminsearch 1', 'Berkeley Madonna 2','fminsearch 2'],columns=['J','b0','bi','k','dlt','p','d','tau'])
# dff['min'] = dff.min(axis=1)
dff

,J,b0,bi,k,dlt,p,d,tau
paper,0.00793,4.090000e-07,2.330000e-07,0.24900,0.77500,14500.00000,0.030000,7.0000
Berkeley Madonna 1,0.00426,2.020900e-07,1.271900e-07,0.41184,1.52453,41085.62299,0.039010,6.0000
fminsearch 1,0.00421,1.997500e-07,1.265000e-07,0.33022,1.51470,40734.00000,0.039136,6.0109
Berkeley Madonna 2,0.00450,2.027300e-07,1.249900e-07,1.81286,1.57038,42987.62471,0.039350,7.0000
fminsearch 2,0.00450,2.026200e-07,1.250600e-07,1.78370,1.57250,42997.00000,0.039383,7.0061
